In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    '../data/Train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'  # Use 'binary' for two classes (caries and gingivitis)
)

test_generator = test_datagen.flow_from_directory(
    '../data/Test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


Found 1486 images belonging to 2 classes.
Found 408 images belonging to 2 classes.


In [2]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to retain its pre-trained features
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification (caries/gingivitis)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [3]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)


C:\Users\isatyamks\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 585ms/step - accuracy: 0.8048 - loss: 0.4340

C:\Users\isatyamks\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


47/47 ━━━━━━━━━━━━━━━━━━━━ 39s 718ms/step - accuracy: 0.8063 - loss: 0.4312 - val_accuracy: 0.9093 - val_loss: 0.2042
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 33s 694ms/step - accuracy: 0.9313 - loss: 0.1802 - val_accuracy: 0.9240 - val_loss: 0.1859
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 34s 730ms/step - accuracy: 0.9305 - loss: 0.1788 - val_accuracy: 0.9044 - val_loss: 0.2095
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 33s 709ms/step - accuracy: 0.9464 - loss: 0.1497 - val_accuracy: 0.9510 - val_loss: 0.1313
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 36s 766ms/step - accuracy: 0.9612 - loss: 0.1207 - val_accuracy: 0.9412 - val_loss: 0.1511
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 38s 802ms/step - accuracy: 0.9465 - loss: 0.1243 - val_accuracy: 0.9216 - val_loss: 0.1549
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 37s 794ms/step - accuracy: 0.9636 - loss: 0.1048 - val_accuracy: 0.9363 - val_loss: 0.1362
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 38s 810ms/step - accuracy: 0.9646 - loss: 0.1008 - val_accuracy: 0.933

In [4]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")


13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 458ms/step - accuracy: 0.9666 - loss: 0.1265
Test accuracy: 0.9583333134651184


In [11]:
from tensorflow.keras.preprocessing import image
import numpy as np
count = 0
for i in range(1,204):
    img = image.load_img(f'../data/Test/Gingivitis/images-{i}.jpg', target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    if prediction > 0.5:
        print("The image is classified as Gingivitis")
        count = count +1
    else:
        print("The image is classified as Caries")
count

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
The image is classified as Gingivitis
1/1 ━━━━━━━━━━━━━━━

188

In [12]:
counti = 0
for i in range(1,204):
    img = image.load_img(f'../data/Test/Caries/images-{i}.jpg', target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    if prediction > 0.5:
        print("The image is classified as Gingivitis")
        
    else:
        print("The image is classified as Caries")
        counti = counti +1
counti

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
The image is classified as Caries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
The image is classified as

201

In [14]:
model.save('..//latest_epoches.keras')
